<a href="https://colab.research.google.com/github/pranao0609/DL_Projects/blob/main/23_NExt_Word_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [9]:
text = "hello how are you hello how have you been hello how is your day"

In [10]:
words = list(set(text.split()))
word2idx = {w: i for i, w in enumerate(words)}
idx2word = {i: w for w, i in word2idx.items()}

sequence_length = 3
data = []
for i in range(len(text.split()) - sequence_length):
    seq = text.split()[i:i + sequence_length]
    target = text.split()[i + sequence_length]
    data.append(( [word2idx[w] for w in seq], word2idx[target]))

In [11]:
X = torch.tensor([x for x, _ in data], dtype=torch.long)
Y = torch.tensor([y for _, y in data], dtype=torch.long)

In [12]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

model = RNNModel(len(words), 10, 20)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [13]:
for epoch in range(200):
    outputs = model(X)
    loss = criterion(outputs, Y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/200], Loss: {loss.item():.4f}')

Epoch [50/200], Loss: 0.0314
Epoch [100/200], Loss: 0.0059
Epoch [150/200], Loss: 0.0034
Epoch [200/200], Loss: 0.0023


In [14]:
with torch.no_grad():
    test_input = torch.tensor([[word2idx["hello"], word2idx["how"], word2idx["is"]]])
    out = model(test_input)
    pred = torch.argmax(out, dim=1).item()
    print("Next word:", idx2word[pred])

Next word: your
